In [2]:
#echo 1 | sudo tee /proc/sys/vm/overcommit_memory
#export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/usr/local/cuda/lib64

import numpy as np

Path = "RBG32Bit"

#Load Training, validation and test images
train_images = np.load(Path + "/train_images.npy")
val_images = np.load(Path + "/val_images.npy")
test_images = np.load(Path + "/test_images.npy")

#Load Training, validation and test labels
train_lab = np.load(Path + "/train_lab.npy")
val_lab = np.load(Path + "/val_lab.npy")
test_lab = np.load(Path + "/test_lab.npy")


In [6]:
from keras import layers
from keras import models
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2
#from keras.models import load_model
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import pickle

os.environ["TF_CPP_MIN_LOG_LEVEL"]="3" 

#*args = 0,3 reguliser params
def create_model(loss, regularizers_n, lr):
    model = models.Sequential()
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(512, activation='relu', kernel_regularizer = l2(regularizers_n)))
    model.add(layers.Dense(10, activation='softmax', kernel_regularizer = l2(regularizers_n)))

    model.compile(loss=loss, optimizer=optimizers.RMSprop(learning_rate = lr), metrics=['acc'])
    return model


model = KerasClassifier(build_fn=create_model, verbose = 2)
lr = [0.001, 0.0005, 0.0001, 0.00005]
epochs = [50]
batches = [64]
loss = ["categorical_crossentropy"]
regularizers_n = [0.001, 0.0005, 0.0001]

param_grid = dict(lr = lr, epochs = epochs, batch_size = batches, loss = loss, regularizers_n = regularizers_n)
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 3)

grid_result = grid.fit(train_images, train_lab, batch_size = 64, validation_data = (val_images, val_lab))


print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))




Epoch 1/50
543/543 - 17s - loss: 1.8467 - acc: 0.3026 - val_loss: 1.7186 - val_acc: 0.3264
Epoch 2/50
543/543 - 17s - loss: 1.6896 - acc: 0.3484 - val_loss: 1.6575 - val_acc: 0.3469
Epoch 3/50
543/543 - 17s - loss: 1.6339 - acc: 0.3667 - val_loss: 1.6682 - val_acc: 0.3522
Epoch 4/50
543/543 - 17s - loss: 1.5976 - acc: 0.3857 - val_loss: 1.5917 - val_acc: 0.3899
Epoch 5/50
543/543 - 17s - loss: 1.5605 - acc: 0.4029 - val_loss: 1.5659 - val_acc: 0.4260
Epoch 6/50
543/543 - 17s - loss: 1.5308 - acc: 0.4248 - val_loss: 1.5759 - val_acc: 0.4088
Epoch 7/50
543/543 - 17s - loss: 1.4974 - acc: 0.4415 - val_loss: 1.5443 - val_acc: 0.4435
Epoch 8/50
543/543 - 17s - loss: 1.4660 - acc: 0.4677 - val_loss: 1.5290 - val_acc: 0.4559
Epoch 9/50
543/543 - 17s - loss: 1.4340 - acc: 0.4850 - val_loss: 1.5973 - val_acc: 0.4370
Epoch 10/50
543/543 - 17s - loss: 1.4042 - acc: 0.5040 - val_loss: 1.5135 - val_acc: 0.4728
Epoch 11/50
543/543 - 17s - loss: 1.3754 - acc: 0.5220 - val_loss: 1.5725 - val_acc: 0.46

Epoch 36/50
543/543 - 17s - loss: 0.9620 - acc: 0.7714 - val_loss: 1.7593 - val_acc: 0.5611
Epoch 37/50
543/543 - 17s - loss: 0.9453 - acc: 0.7745 - val_loss: 1.7278 - val_acc: 0.5717
Epoch 38/50
543/543 - 17s - loss: 0.9339 - acc: 0.7794 - val_loss: 1.7664 - val_acc: 0.5730
Epoch 39/50
543/543 - 17s - loss: 0.9363 - acc: 0.7794 - val_loss: 1.8812 - val_acc: 0.5736
Epoch 40/50
543/543 - 17s - loss: 0.9204 - acc: 0.7872 - val_loss: 1.7695 - val_acc: 0.5751
Epoch 41/50
543/543 - 17s - loss: 0.9181 - acc: 0.7870 - val_loss: 1.7529 - val_acc: 0.5799
Epoch 42/50
543/543 - 17s - loss: 0.9087 - acc: 0.7921 - val_loss: 1.7347 - val_acc: 0.5780
Epoch 43/50
543/543 - 17s - loss: 0.9028 - acc: 0.7945 - val_loss: 1.8311 - val_acc: 0.5673
Epoch 44/50
543/543 - 17s - loss: 0.9058 - acc: 0.7941 - val_loss: 1.8814 - val_acc: 0.5588
Epoch 45/50
543/543 - 17s - loss: 0.8936 - acc: 0.8032 - val_loss: 1.8374 - val_acc: 0.5763
Epoch 46/50
543/543 - 17s - loss: 0.8841 - acc: 0.8033 - val_loss: 1.7563 - val_

Epoch 21/50
543/543 - 17s - loss: 1.0440 - acc: 0.7249 - val_loss: 1.5823 - val_acc: 0.5799
Epoch 22/50
543/543 - 17s - loss: 1.0356 - acc: 0.7294 - val_loss: 1.6526 - val_acc: 0.5650
Epoch 23/50
543/543 - 17s - loss: 1.0182 - acc: 0.7395 - val_loss: 1.6313 - val_acc: 0.5859
Epoch 24/50
543/543 - 17s - loss: 1.0020 - acc: 0.7468 - val_loss: 1.6105 - val_acc: 0.5736
Epoch 25/50
543/543 - 17s - loss: 0.9901 - acc: 0.7541 - val_loss: 1.8460 - val_acc: 0.5452
Epoch 26/50
543/543 - 17s - loss: 0.9786 - acc: 0.7621 - val_loss: 1.6228 - val_acc: 0.5846
Epoch 27/50
543/543 - 17s - loss: 0.9679 - acc: 0.7669 - val_loss: 1.6138 - val_acc: 0.5969
Epoch 28/50
543/543 - 17s - loss: 0.9590 - acc: 0.7703 - val_loss: 1.6908 - val_acc: 0.5952
Epoch 29/50
543/543 - 17s - loss: 0.9476 - acc: 0.7771 - val_loss: 1.7534 - val_acc: 0.5842
Epoch 30/50
543/543 - 17s - loss: 0.9314 - acc: 0.7817 - val_loss: 1.7794 - val_acc: 0.5836
Epoch 31/50
543/543 - 17s - loss: 0.9305 - acc: 0.7844 - val_loss: 1.7960 - val_

Epoch 6/50
543/543 - 17s - loss: 1.4595 - acc: 0.4766 - val_loss: 1.6273 - val_acc: 0.4238
Epoch 7/50
543/543 - 17s - loss: 1.4122 - acc: 0.5048 - val_loss: 1.4740 - val_acc: 0.4794
Epoch 8/50
543/543 - 17s - loss: 1.3715 - acc: 0.5328 - val_loss: 1.4861 - val_acc: 0.4945
Epoch 9/50
543/543 - 17s - loss: 1.3291 - acc: 0.5562 - val_loss: 1.5189 - val_acc: 0.4989
Epoch 10/50
543/543 - 17s - loss: 1.2865 - acc: 0.5859 - val_loss: 1.4645 - val_acc: 0.5224
Epoch 11/50
543/543 - 17s - loss: 1.2549 - acc: 0.6031 - val_loss: 1.6612 - val_acc: 0.5132
Epoch 12/50
543/543 - 17s - loss: 1.2150 - acc: 0.6292 - val_loss: 1.4992 - val_acc: 0.5369
Epoch 13/50
543/543 - 17s - loss: 1.1842 - acc: 0.6462 - val_loss: 1.5823 - val_acc: 0.5344
Epoch 14/50
543/543 - 17s - loss: 1.1602 - acc: 0.6606 - val_loss: 1.5933 - val_acc: 0.5502
Epoch 15/50
543/543 - 17s - loss: 1.1343 - acc: 0.6780 - val_loss: 1.5682 - val_acc: 0.5596
Epoch 16/50
543/543 - 17s - loss: 1.1037 - acc: 0.6931 - val_loss: 1.4890 - val_acc:

Epoch 43/50
543/543 - 17s - loss: 0.5871 - acc: 0.8936 - val_loss: 2.2140 - val_acc: 0.6078
Epoch 44/50
543/543 - 17s - loss: 0.5768 - acc: 0.8971 - val_loss: 2.1858 - val_acc: 0.5753
Epoch 45/50
543/543 - 17s - loss: 0.5782 - acc: 0.8956 - val_loss: 2.3058 - val_acc: 0.6114
Epoch 46/50
543/543 - 17s - loss: 0.5741 - acc: 0.8983 - val_loss: 2.1390 - val_acc: 0.6055
Epoch 47/50
543/543 - 17s - loss: 0.5796 - acc: 0.8977 - val_loss: 2.0000 - val_acc: 0.6017
Epoch 48/50
543/543 - 17s - loss: 0.5675 - acc: 0.9001 - val_loss: 2.1363 - val_acc: 0.5948
Epoch 49/50
543/543 - 17s - loss: 0.5611 - acc: 0.9000 - val_loss: 2.1892 - val_acc: 0.5962
Epoch 50/50
543/543 - 17s - loss: 0.5633 - acc: 0.8996 - val_loss: 2.0134 - val_acc: 0.5972
272/272 - 3s - loss: 0.7395 - acc: 0.8750
Epoch 1/50
543/543 - 17s - loss: 1.7814 - acc: 0.3125 - val_loss: 1.6546 - val_acc: 0.3679
Epoch 2/50
543/543 - 17s - loss: 1.6307 - acc: 0.3651 - val_loss: 1.5850 - val_acc: 0.3896
Epoch 3/50
543/543 - 17s - loss: 1.5624 

...

Epoch 9/50
543/543 - 17s - loss: 1.7471 - acc: 0.3512 - val_loss: 1.7824 - val_acc: 0.3327
Epoch 10/50
543/543 - 17s - loss: 1.7348 - acc: 0.3528 - val_loss: 1.7261 - val_acc: 0.3539
Epoch 11/50
543/543 - 17s - loss: 1.7231 - acc: 0.3584 - val_loss: 1.7244 - val_acc: 0.3553
Epoch 12/50
543/543 - 17s - loss: 1.7116 - acc: 0.3602 - val_loss: 1.7099 - val_acc: 0.3614
Epoch 13/50
543/543 - 17s - loss: 1.7022 - acc: 0.3611 - val_loss: 1.7130 - val_acc: 0.3447
Epoch 14/50
543/543 - 17s - loss: 1.6928 - acc: 0.3626 - val_loss: 1.6868 - val_acc: 0.3814
Epoch 15/50
543/543 - 17s - loss: 1.6842 - acc: 0.3693 - val_loss: 1.6816 - val_acc: 0.3739
Epoch 16/50
543/543 - 17s - loss: 1.6781 - acc: 0.3708 - val_loss: 1.6894 - val_acc: 0.3753
Epoch 17/50
543/543 - 17s - loss: 1.6681 - acc: 0.3730 - val_loss: 1.6715 - val_acc: 0.3724
Epoch 18/50
543/543 - 17s - loss: 1.6599 - acc: 0.3793 - val_loss: 1.6646 - val_acc: 0.3639
Epoch 19/50
543/543 - 17s - loss: 1.6540 - acc: 0.3800 - val_loss: 1.6536 - val_a

Epoch 48/50
543/543 - 17s - loss: 1.4503 - acc: 0.4729 - val_loss: 1.5185 - val_acc: 0.4536
Epoch 49/50
543/543 - 17s - loss: 1.4500 - acc: 0.4755 - val_loss: 1.5278 - val_acc: 0.4470
Epoch 50/50
543/543 - 17s - loss: 1.4384 - acc: 0.4757 - val_loss: 1.5174 - val_acc: 0.4585
272/272 - 3s - loss: 1.4718 - acc: 0.4618
Epoch 1/50
543/543 - 17s - loss: 2.3683 - acc: 0.2334 - val_loss: 2.0190 - val_acc: 0.2783
Epoch 2/50
543/543 - 17s - loss: 1.9331 - acc: 0.3059 - val_loss: 1.8789 - val_acc: 0.3101
Epoch 3/50
543/543 - 16s - loss: 1.8578 - acc: 0.3239 - val_loss: 1.8411 - val_acc: 0.3280
Epoch 4/50
543/543 - 17s - loss: 1.8200 - acc: 0.3332 - val_loss: 1.7977 - val_acc: 0.3382
Epoch 5/50
543/543 - 17s - loss: 1.7903 - acc: 0.3357 - val_loss: 1.7614 - val_acc: 0.3556
Epoch 6/50
543/543 - 17s - loss: 1.7671 - acc: 0.3416 - val_loss: 1.7478 - val_acc: 0.3449
Epoch 7/50
543/543 - 17s - loss: 1.7480 - acc: 0.3483 - val_loss: 1.7388 - val_acc: 0.3458
Epoch 8/50
543/543 - 17s - loss: 1.7308 - acc

Epoch 37/50
543/543 - 17s - loss: 1.5486 - acc: 0.4231 - val_loss: 1.5804 - val_acc: 0.4112
Epoch 38/50
543/543 - 17s - loss: 1.5434 - acc: 0.4241 - val_loss: 1.5887 - val_acc: 0.4080
Epoch 39/50
543/543 - 17s - loss: 1.5374 - acc: 0.4280 - val_loss: 1.5783 - val_acc: 0.4074
Epoch 40/50
543/543 - 17s - loss: 1.5313 - acc: 0.4317 - val_loss: 1.5704 - val_acc: 0.4103
Epoch 41/50
543/543 - 17s - loss: 1.5236 - acc: 0.4363 - val_loss: 1.5590 - val_acc: 0.4217
Epoch 42/50
543/543 - 17s - loss: 1.5199 - acc: 0.4373 - val_loss: 1.5789 - val_acc: 0.4204
Epoch 43/50
543/543 - 17s - loss: 1.5127 - acc: 0.4435 - val_loss: 1.5844 - val_acc: 0.4126
Epoch 44/50
543/543 - 17s - loss: 1.5078 - acc: 0.4439 - val_loss: 1.5619 - val_acc: 0.4230
Epoch 45/50
543/543 - 17s - loss: 1.5019 - acc: 0.4459 - val_loss: 1.5665 - val_acc: 0.4210
Epoch 46/50
543/543 - 17s - loss: 1.4968 - acc: 0.4483 - val_loss: 1.5346 - val_acc: 0.4410
Epoch 47/50
543/543 - 17s - loss: 1.4889 - acc: 0.4521 - val_loss: 1.5306 - val_

Epoch 24/50
543/543 - 17s - loss: 1.5760 - acc: 0.4032 - val_loss: 1.5814 - val_acc: 0.3972
Epoch 25/50
543/543 - 17s - loss: 1.5674 - acc: 0.4081 - val_loss: 1.5684 - val_acc: 0.4094
Epoch 26/50
543/543 - 17s - loss: 1.5606 - acc: 0.4089 - val_loss: 1.5619 - val_acc: 0.4043
Epoch 27/50
543/543 - 17s - loss: 1.5570 - acc: 0.4112 - val_loss: 1.5509 - val_acc: 0.4114
Epoch 28/50
543/543 - 17s - loss: 1.5479 - acc: 0.4142 - val_loss: 1.5595 - val_acc: 0.4054
Epoch 29/50
543/543 - 17s - loss: 1.5419 - acc: 0.4184 - val_loss: 1.5485 - val_acc: 0.4149
Epoch 30/50
543/543 - 17s - loss: 1.5367 - acc: 0.4191 - val_loss: 1.5382 - val_acc: 0.4161
Epoch 31/50
543/543 - 17s - loss: 1.5302 - acc: 0.4245 - val_loss: 1.5460 - val_acc: 0.4123
Epoch 32/50
543/543 - 17s - loss: 1.5238 - acc: 0.4239 - val_loss: 1.5336 - val_acc: 0.4295
Epoch 33/50
543/543 - 17s - loss: 1.5154 - acc: 0.4302 - val_loss: 1.5260 - val_acc: 0.4249
Epoch 34/50
543/543 - 17s - loss: 1.5095 - acc: 0.4359 - val_loss: 1.5427 - val_

Epoch 11/50
814/814 - 25s - loss: 1.0561 - acc: 0.6602 - val_loss: 1.4033 - val_acc: 0.5697
Epoch 12/50
814/814 - 25s - loss: 0.9924 - acc: 0.6911 - val_loss: 1.4246 - val_acc: 0.5702
Epoch 13/50
814/814 - 25s - loss: 0.9299 - acc: 0.7227 - val_loss: 1.4444 - val_acc: 0.5803
Epoch 14/50
814/814 - 25s - loss: 0.8730 - acc: 0.7495 - val_loss: 1.4665 - val_acc: 0.5937
Epoch 15/50
814/814 - 26s - loss: 0.8170 - acc: 0.7769 - val_loss: 1.6186 - val_acc: 0.5939
Epoch 16/50
814/814 - 25s - loss: 0.7698 - acc: 0.7948 - val_loss: 1.5999 - val_acc: 0.6088
Epoch 17/50
814/814 - 25s - loss: 0.7281 - acc: 0.8159 - val_loss: 1.5806 - val_acc: 0.6121
Epoch 18/50
814/814 - 25s - loss: 0.6954 - acc: 0.8292 - val_loss: 1.7420 - val_acc: 0.6031
Epoch 19/50
814/814 - 25s - loss: 0.6639 - acc: 0.8437 - val_loss: 1.6830 - val_acc: 0.6147
Epoch 20/50
814/814 - 25s - loss: 0.6365 - acc: 0.8533 - val_loss: 1.7275 - val_acc: 0.6163
Epoch 21/50
814/814 - 25s - loss: 0.6105 - acc: 0.8648 - val_loss: 1.7814 - val_